<a href="https://colab.research.google.com/github/goranagojic/paralelno-racunarstvo/blob/main/PR_termin7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paralelno računarstvo - Vežba 7

Priprema za Z2.

Vežba obuhvata zadatke:
*    Zadatak 1 - traženje para najbližih tačaka u ravni

## Preduslovi

Pre nego započnete implementaciju zadatka, **obavezno** izvršite kod iz ćelije ispod. Kod će:

*   proveriti da li je na sistemu instalirana biblioteka `libgomp` potrebna za izvršavanje OpenMP programa i ako nije instaliraće je,
*   napraviti direktorijum u koji će se čuvati sve izvrne i izvršne datoteke
*   preuzeti datoteku `hpc_helpers.hpp` i sačuvati je na putanju /home/openmp/

In [ ]:
%%bash

# instaliraj libgomp biblioteku ukoliko vec nije instalirana
dpkg -l | grep libgomp1
[[ $? -ne 0 ]] && sudo apt-get install libgomp1 || echo "Biblioteka libgomp1 je već instalirana."

# napravi radni direktorijum
mkdir -p /home/openmp

# preuzmi hpc_helper zaglavlje
[[ -f hpc_helpers.zip ]] || wget -O hpc_helpers.zip "https://drive.google.com/u/0/uc?id=1PeEfh8h5SjEcQ5154Z6TpHIlJogQODwM&export=download"
unzip -oqqd /home/openmp/ hpc_helpers.zip


Po završetku prethodne ćelije možete pokrentui narednu koja proverava da li je hpc_helpers zaglavlje raspakovano na željenu putanju. Ukoliko je sve u redu, naredba će ispisati absolutnu putanju do zaglavlja.

In [ ]:
!ls /home/openmp/hpc_helpers.hpp

## Zadatak 1
Implementirati rešenje koje za zadati niz planarnih tačaka pronalazi najbliži par tačaka (eng. *closest pair of points problem*). Rastojanje između tačaka je definisano kao Euklidsko rastojanje i za svake dve tačke $p$ i $q$ se može sračunati kao:

$$
d = \sqrt{(p_{x} - q_{x})^2 + (p_{y} - q_{y})^2}
$$

Data je postavka zadatka. Potrebno je:
*    Implementirati funkciju `find_closest` tako da sadrži sekvencijalno rešenje problema. Dovoljno je implementirati *brute-force* rešenje za traženje najbližeg para. Obratiti pažnju na vreme izvršavanja.
*    Implementirati funkciju `find_closest_parallel` tako da sadrži inicijalno OpenMp rešenje problema. Obratiti pažnju na vreme izvršavnaja i uporediti ga sa vremenom izvršavanja sekvencijalnog rešenja. Varirati broj tačaka pri testiranju.
*    Razmisliti o optimizacijama OpenMP rešenja koje ste napisali (npr. isprobati različite režime raspoređivanja posla po nitima). U ćeliju ispod postavke zadatka zapisati modifikacije isprobane nad inicijalnim OpenMP rešenjem i zapažanja kako su modifikacije uticale na tačnost rezultata i vreme izvršavanja. Probajte da objasnite zašto su primenjene modifikacije tako uticale na osnovno rešenje.

In [ ]:
%%writefile /home/openmp/closestpointspair.cpp

#include <iostream>
#include <vector>
#include <algorithm>
#include <random>
#include <cassert>

#include "hpc_helpers.hpp"


#define NOTIMPLEMENTED std::pair<Point, Point>()


const uint32_t NUM_POINTS = 100000; // menjati broj tacaka u nizu i posmatrati kako se ponasaju vremena izvrsavanja resenja


struct Point {
    int32_t x;
    int32_t y;

    Point() {}
    Point(int32_t x_, int32_t y_) : x(x_), y(y_) {}
};


/**
 * Implementirati C++ sekvencijalno resenje.
 */ 
std::pair<Point, Point> find_closest(const std::vector<Point> points) {
    // TODO implementirati

    return NOTIMPLEMENTED; // modifikovati povratnu vrednost
}

/**
 *  Implementirati C++ OpenMP paralelno resenje.
 */ 
std::pair<Point, Point> find_closest_parallel(const std::vector<Point> points) {
    // TODO implementirati

    return NOTIMPLEMENTED; // modifikovati povratnu vrednost
}


/**
 * Inicijalizacija vektora razlicitim tackama u istoj ravni. Ukoliko vektor vec
 * ima elemenata, novi elementi ce biti dodati na postojece.
 */
void init_points(std::vector<Point> &points) {
    
    std::vector<int32_t> coords(NUM_POINTS * 2);

    // generise NUM_POINTS * 2 razlicitih vrednosti koje ce biti koordinate
    int32_t counter = 0;
    std::generate(coords.begin(), coords.end(), [&counter](){ return counter++; });

    // mesa elemente vektora koordinata
    std::random_device rd;
    auto g = std::mt19937(rd());
    std::shuffle(coords.begin(), coords.end(), g);

    // pravi tacke od parova susednih koordinata, sve tacke ce biti razlicite
    for (auto it = coords.begin(); it != coords.end(); it += 2) {
        points.emplace_back(*it, *(it+1));  // x, y
    }
    assert(points.size() * 2 == coords.size());
}


int main() {

    TIMERSTART(alloc)
    std::vector<Point> points;
    TIMERSTOP(alloc)

    TIMERSTART(init)
    init_points(points);
    TIMERSTOP(init)

    TIMERSTART(seq)
    auto closest = find_closest(points);
    TIMERSTOP(seq)
    
    std::cout << "Najblizi par (" << closest.first.x << ", " << closest.first.y << "), ";
    std::cout << "(" << closest.second.x << ", " << closest.second.y << ")" << std::endl;

    TIMERSTART(openmp)
    auto closest2 = find_closest_parallel(points);
    TIMERSTOP(openmp)

    std::cout << "Najblizi par (" << closest2.first.x << ", " << closest2.first.y << "), ";
    std::cout << "(" << closest2.second.x << ", " << closest2.second.y << ")" << std::endl;
    
}

### Kompajliranje izvornog koda

Pokrenite komandu u ćeliji ispod kako biste kompajlirali rešenje.

In [ ]:
%%bash

cd /home/openmp
g++ hpc_helpers.hpp closestpointspair.cpp -o closestpointspair -fopenmp

### [Opciono] Provera da li postoji izvršna datoteka

In [ ]:
![[ -f /home/openmp/closestpointspair ]] && echo "Postoji." || echo "Ne postoji."

### Pokretanje rešenja

Za promenu broja tačaka promeniti vrednost konstante `NUM_POINTS.`

In [ ]:
%%bash

cd /home/openmp
./closestpointspair

## Zapažanja

Ovde zapisati isprobane optimizacije i njihove efekte na izvršavanje rešenja.